In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
df = pd.read_csv("/content/drive/MyDrive/Machine learning/csv/dermatology.csv",header=None)
print(type(df))

new_row=['Erythema','Scaling','Definite borders','Itching','Koebner Phenomenon','Polygonal papules','Follicular papules','Oral mucosal involvement','Knee and elbow involvement',
'Scalp involvement','Family history','Melanin Incontinence','Eosinophils in the infiltrate','PNL infiltrate','Fibrosis of the papillary dermis','Exocytosis','Acanthosis',
'Hyperkeratosis','Parakeratosis','Clubbing of the rete ridges','Elongation of the rete ridges','Thinning of the suprapapillary epidermis','Spongiform pustule','Munro microabcess',
'Focal hypergranulosis','Disappearance of the granular layer','Vacuolisation and damage of basal layer','Spongiosis','Saw-tooth appearance of retes','Follicular horn plug',
'Perifollicular parakeratosis','Inflammatory monoluclear inflitrate','Band-like infiltrate','Age','Class code']
df.columns=new_row
df=df.replace('?',np.nan)

# changing the datatype of Age attribute as it in string
df['Age']=df['Age'].astype('float64')
df=df.fillna(df.mean())
print(df['Age'].mean())
print(df.info())

<class 'pandas.core.frame.DataFrame'>
36.29608938547486
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 366 entries, 0 to 365
Data columns (total 35 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   Erythema                                  366 non-null    int64  
 1   Scaling                                   366 non-null    int64  
 2   Definite borders                          366 non-null    int64  
 3   Itching                                   366 non-null    int64  
 4   Koebner Phenomenon                        366 non-null    int64  
 5   Polygonal papules                         366 non-null    int64  
 6   Follicular papules                        366 non-null    int64  
 7   Oral mucosal involvement                  366 non-null    int64  
 8   Knee and elbow involvement                366 non-null    int64  
 9   Scalp involvement                         366 non-nul

In [ ]:
from pandas.core.arrays import categorical
#dividing the data into training and testing
print(df.shape)
n=df.shape[0]
print(n)
# our training data
train_df=df.sample(int(0.7*n))
# our testing data
test_df=df.drop(train_df.index)
print(train_df.shape)
print(test_df.shape)

(366, 35)
366
(256, 35)
(110, 35)


In [ ]:
class_values = train_df['Class code'].unique()
class_values.sort()
priori = {}
for i in class_values:
  priori[i]=train_df[train_df['Class code']==i].shape[0]/train_df.shape[0]
print(priori) 

{1: 0.31640625, 2: 0.171875, 3: 0.1875, 4: 0.1328125, 5: 0.13671875, 6: 0.0546875}


In [ ]:
attribute = train_df.columns[:-2]
likelihood ={}
for i in attribute :
  values=train_df[i].unique()
  values.sort()
  att_values=['Class code']+ list(values)
  att_df=pd.DataFrame(columns=att_values)
  for j in class_values:
    class_d = {}
    class_d['Class code'] = j
    for k in values:
      num = train_df[(train_df['Class code']==j) & (train_df[i]==k)].shape[0]
      den = train_df[train_df['Class code']==j].shape[0]
      class_d[k] = num/den
    att_df = att_df.append(class_d, ignore_index=True)
  likelihood[i] = att_df
print(likelihood['Erythema'])


   Class code         0         1         2         3
0         1.0  0.012346  0.061728  0.592593  0.333333
1         2.0  0.000000  0.090909  0.568182  0.340909
2         3.0  0.020833  0.104167  0.687500  0.187500
3         4.0  0.000000  0.235294  0.676471  0.088235
4         5.0  0.057143  0.485714  0.371429  0.085714
5         6.0  0.000000  0.142857  0.785714  0.071429


In [ ]:
row = test_df.iloc[34:35].to_dict(orient = 'records')[0]
print(row)

{'Erythema': 1, 'Scaling': 1, 'Definite borders': 2, 'Itching': 2, 'Koebner Phenomenon': 2, 'Polygonal papules': 0, 'Follicular papules': 2, 'Oral mucosal involvement': 0, 'Knee and elbow involvement': 1, 'Scalp involvement': 2, 'Family history': 0, 'Melanin Incontinence': 0, 'Eosinophils in the infiltrate': 0, 'PNL infiltrate': 1, 'Fibrosis of the papillary dermis': 0, 'Exocytosis': 0, 'Acanthosis': 2, 'Hyperkeratosis': 1, 'Parakeratosis': 2, 'Clubbing of the rete ridges': 3, 'Elongation of the rete ridges': 3, 'Thinning of the suprapapillary epidermis': 3, 'Spongiform pustule': 2, 'Munro microabcess': 2, 'Focal hypergranulosis': 0, 'Disappearance of the granular layer': 0, 'Vacuolisation and damage of basal layer': 0, 'Spongiosis': 0, 'Saw-tooth appearance of retes': 0, 'Follicular horn plug': 0, 'Perifollicular parakeratosis': 0, 'Inflammatory monoluclear inflitrate': 2, 'Band-like infiltrate': 0, 'Age': 55.0, 'Class code': 1}


In [ ]:
# Testing our modle
count =0
new_c = []
for i in range(test_df.shape[0]):
  test_row = test_df.iloc[i:i+1,:].to_dict(orient = 'records')[0]
  code = 1
  max_prob = 0
  for j in class_values:
    prob = priori[j]
    for k in attribute:
      att_df = likelihood[k]
      att_df_row = att_df.loc[att_df['Class code']==j]
      prob = prob*float(att_df_row[test_row[k]])
    if prob >max_prob:
      max_prob = prob
      code = j
  new_c.append(code)
  if test_row['Class code']==code:
    count = count +1
test_df['Predicted Class'] = new_c
print(test_df)

     Erythema  Scaling  Definite borders  Itching  Koebner Phenomenon  \
6           2        1                 0        2                   0   
21          3        2                 2        0                   0   
23          2        2                 2        3                   2   
24          2        0                 0        3                   0   
27          1        2                 2        3                   0   
..        ...      ...               ...      ...                 ...   
354         2        0                 1        2                   0   
358         2        3                 2        1                   0   
361         2        1                 1        0                   1   
363         3        2                 2        2                   3   
365         3        2                 2        0                   0   

     Polygonal papules  Follicular papules  Oral mucosal involvement  \
6                    0                   0         

In [ ]:
print(count)
print(len(test_df))
print(count/test_df.shape[0])

96
110
0.8727272727272727


In [ ]:
value = test_df[test_df['Class code'] != test_df['Predicted Class']].shape[0]
print(value)
error = value/len(test_df)
print("Accuracy : ",1-error)

14
Accuracy :  0.8727272727272728
